<a href="https://colab.research.google.com/github/John-zf/SMAML_for_Tool_Wear_Monitoring-Prediction/blob/master/SMAML_for_Experiments_c1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import torch

print(torch.cuda.get_device_name(0))
drive.mount('/content/drive/', force_remount=True)

Tesla T4
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
Tesla T4
Mounted at /content/drive/


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Load data and transform them into 2d images
import pandas as pd
import numpy as np
import os

def get_data(data_folder):
  # Read the sensor data
  data = []
  for file in os.listdir(data_folder):
    file_path = data_folder + file
    ins = pd.read_csv(file_path, header=None)
    ins = np.array(ins).T[:, :224*224]
    # Centralization
    ins = np.array([x-np.mean(x) for x in ins])
    data.append(ins)
  data = np.array(data)
  print('{} reading is done.'.format(data_folder))
  return data

data_folder_c1 = '/content/drive/My Drive/mhm_data/c1/c1/'
data_folder_c4 = '/content/drive/My Drive/mhm_data/c4/c4/'
data_folder_c6 = '/content/drive/My Drive/mhm_data/c6/c6/'

data_c1 = get_data(data_folder_c1)
data_c4 = get_data(data_folder_c4)
data_c6 = get_data(data_folder_c6) 

/content/drive/My Drive/mhm_data/c1/c1/ reading is done.
/content/drive/My Drive/mhm_data/c4/c4/ reading is done.
/content/drive/My Drive/mhm_data/c6/c6/ reading is done.


In [ ]:
# Load target
def get_target(target_file, step=10):
    # Read target, 
    target = pd.read_csv(target_file)
    target = np.array(target[['flute_1', 'flute_2', 'flute_3']])
    target = [x[0] for x in target]
    print(target)
    if step == 1:
      return target
    else:
      targets_list = []
      for i, value in enumerate(target):
        targets = []
        for j in range(i, (i+step)):
          if j > len(target)-1:
            targets.append(target[-1])
          else:
            targets.append(target[j])
        targets_list.append(targets)
      return np.array(targets_list)

target_file_c1 = '/content/drive/My Drive/mhm_data/c1/c1_wear.csv'
target_file_c4 = '/content/drive/My Drive/mhm_data/c4/c4_wear.csv'
target_file_c6 = '/content/drive/My Drive/mhm_data/c6/c6_wear.csv'

target_c1 = get_target(target_file_c1)
target_c4 = get_target(target_file_c4)
target_c6 = get_target(target_file_c6)

[32.31711361, 37.914879, 43.08790971, 47.8590723, 52.25032922, 56.28276562, 59.97661561, 63.351287899999996, 66.4253909, 69.21675719, 71.74246752, 74.01887416, 76.06162381, 77.88567991, 79.50534445, 80.93427925, 82.18552678, 83.27153045, 84.20415441, 84.99470288, 85.65393900000001, 86.19210327, 86.61893145, 86.94367205, 87.17510345, 87.32155046, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87

In [ ]:
# split data

######## Training and Test Data ########
train_fea = [data_c4, data_c6]
train_tar = [target_c4, target_c6]
test_fea = data_c1
test_tar = target_c1
#############################################

train_data = np.vstack(train_fea)
train_label = np.vstack(train_tar)

In [ ]:
# Traditional feature extraction
import numpy as np
import scipy.stats as sts
from pywt import WaveletPacket

def rms_fea(a):
	return np.sqrt(np.mean(np.square(a)))

def var_fea(a):
	return np.var(a)

def max_fea(a):
	return np.max(a)

def pp_fea(a):
	return np.max(a)-np.min(a)

def skew_fea(a):
	return sts.skew(a)

def kurt_fea(a):
	return sts.kurtosis(a)

def wave_fea(a):
	wp = WaveletPacket(a,'db1', maxlevel=8)
	nodes = wp.get_level(8, "freq")
	return np.linalg.norm(np.array([n.data for n in nodes]), 2)

def spectral_kurt(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1: int(N/2)] * 2.00/N
	return sts.kurtosis(mag)

def spectral_skw(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1:int(N/2)]*2.00/N
	return sts.skew(mag)

def spectral_pow(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1:int(N/2)]*2.00/N
	return np.mean(np.power(mag, 3))
 
def extract_fea(data):
	data_fea = []
	for i in range(7):
		data_fea.append(rms_fea(data[i]))
		data_fea.append(var_fea(data[i]))
		data_fea.append(max_fea(data[i]))
		data_fea.append(pp_fea(data[i]))
		data_fea.append(skew_fea(data[i]))
		data_fea.append(kurt_fea(data[i]))
		data_fea.append(wave_fea(data[i]))
		data_fea.append(spectral_kurt(data[i]))
		data_fea.append(spectral_skw(data[i]))
		data_fea.append(spectral_pow(data[i]))
	return np.array(data_fea)
 
trian_data_ml = np.array([extract_fea(x) for x in train_data])
test_data_ml = np.array([extract_fea(x) for x in test_fea])

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

setp = 9
train_label_ml = train_label[:, setp]
test_label_ml = test_tar[:, setp]

scaler = StandardScaler()
scaler.fit(trian_data_ml)
train_data_sc = scaler.transform(trian_data_ml)
test_data_sc = scaler.transform(test_data_ml)

# SVR
regr = SVR()
regr.fit(train_data_sc, train_label_ml)
preds = regr.predict(test_data_sc)
print('SVR -> MSE:{:.2f}, MAE:{:.2f}'.format(mean_squared_error(test_label_ml, preds), mean_absolute_error(test_label_ml, preds)))

# XGBoost
seed = 2020
for i in range(10):
  regr = XGBRegressor(
      learning_rate=0.1,
      n_estimators=200,
      max_depth=1,
      subsample=0.8,
      colsample_bytree=0.8,
      objective='reg:squarederror',
      random_state=seed+i)
  regr.fit(train_data_sc, train_label_ml)
  preds = regr.predict(test_data_sc)
  print('{:.2f},{:.2f}'.format(mean_squared_error(test_label_ml, preds), mean_absolute_error(test_label_ml, preds)))

SVR -> MSE:157.26, MAE:8.22
104.73,8.21
96.34,7.99
95.17,7.99
108.86,8.21
122.76,8.85
96.33,7.87
100.51,7.94
100.91,7.87
113.57,8.47
104.14,8.06


In [ ]:
# For Pytorch
import random
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split


def seed_torch(seed=2020):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
seed_torch()

train_data_dl = torch.from_numpy(train_data).float()
train_label_dl = torch.from_numpy(train_label).float()
deal_dataset = TensorDataset(train_data_dl, train_label_dl)

# Split training and validation data
ratio = 0.9
num_train = int(ratio * len(deal_dataset))
num_val = len(deal_dataset) - num_train
train_data_dl, val_data_dl = random_split(deal_dataset, [num_train, num_val])
print('train:', len(train_data_dl), 'validation:', len(val_data_dl))
train_loader = DataLoader(dataset=train_data_dl, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data_dl, batch_size=16, shuffle=True)

# Test data
test_data_dl = torch.from_numpy(test_fea).float()
test_label_dl = torch.from_numpy(test_tar).float()

deal_test = TensorDataset(test_data_dl, test_label_dl)
test_loader = DataLoader(dataset=deal_test, batch_size=16, shuffle=False)
print('test:', len(deal_test))

train: 567 validation: 63
test: 315


In [ ]:
# Define Mono Loss Function
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class MonoLossFunc(nn.Module):
  def __init__(self, lamb=1):
    super(MonoLossFunc, self).__init__()
    self.lamb = lamb

  def forward(self, prediction, truth):
    # calculate mse
    mse_func = torch.nn.MSELoss(reduction='sum')
    loss1 = mse_func(prediction, truth)

    # calculate monotonicity
    data1 = prediction[:, : -1]
    data2 = prediction[:, 1 :]
    loss2 = torch.sum(torch.clamp(data1 - data2, min=0))
    return loss1 + self.lamb * loss2


def evaluate(model, hidden, criteria, val_loader, test_loader):
  model.eval()
  val_loss = 0.0
  # hidden = model.hidden
  hidden = model.initHidden()
  for i, (inputs, labels) in enumerate(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, _ = model(inputs, hidden)
    loss = criteria(output, labels)
    val_loss += loss.item()
  val_loss = val_loss/63

  mse_0 = 0.0
  mae_0 = 0.0
  mse_1 = 0.0
  mae_1 = 0.0
  mse_5 = 0.0
  mae_5 = 0.0
  mse_9 = 0.0
  mae_9 = 0.0

  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, weights = model(inputs, hidden)
    mse_0 += torch.sum(torch.pow((output[:, 0] - labels[:, 0]), 2)).item()
    mae_0 += torch.sum(torch.abs(output[:, 0] - labels[:, 0])).item()

    mse_1 += torch.sum(torch.pow((output[:, 1] - labels[:, 1]), 2)).item()
    mae_1 += torch.sum(torch.abs(output[:, 1] - labels[:, 1])).item()

    mse_5 += torch.sum(torch.pow((output[:, 5] - labels[:, 5]), 2)).item()
    mae_5 += torch.sum(torch.abs(output[:, 5] - labels[:, 5])).item()

    mse_9 += torch.sum(torch.pow((output[:, 9] - labels[:, 9]), 2)).item()
    mae_9 += torch.sum(torch.abs(output[:, 9] - labels[:, 9])).item()

  return val_loss, weights, mse_0/315, mae_0/315, mse_1/315, mae_1/315,\
      mse_5/315, mae_5/315, mse_9/315, mae_9/315


def train(model, lamb=0, max_iter=200):
  learning_rate = 0.001
  model.to(device)
  # hidden = model.hidden

  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.9)
  criteria = MonoLossFunc(lamb=lamb)

  # training the model
  for epoch in range(max_iter):
    model.train()

    hidden = model.initHidden()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      output, _ = model(inputs, hidden)
      loss = criteria(output, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
    train_loss = train_loss/567
    val_loss, weights, mse_0, mae_0, mse_1, mae_1, mse_5, mae_5, \
    mse_9, mae_9 = evaluate(model, hidden, criteria, val_loader, test_loader)
    print('EPOCH={} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}\
    {:.6f} {:.6f} {:.6f}'.format(epoch+1, train_loss, val_loss, \
    mse_0, mae_0, mse_1, mae_1, mse_5, mae_5, mse_9, mae_9))

In [ ]:
# CNN
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
          nn.Linear(256*10, 10),
        )

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    output = self.encoder(input)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 27683.128210 36688.550595 627.885124 21.993053 2091.266655 35.938150 1800.562021    34.497418 2406.853189 39.389100
EPOCH=2 5340.267985 9899.740079 117.201517 8.187642 358.714604 14.982607 155.234696    10.207590 345.865407 14.627842
EPOCH=3 3577.264736 4513.816716 1056.789618 26.609429 642.008478 22.275126 870.207278    25.326321 761.157626 24.945435
EPOCH=4 3504.846712 1507.063957 777.813053 25.245856 648.743756 24.848861 580.305469    21.914392 497.630811 21.522438
EPOCH=5 7960.360112 6094.723834 156.329126 10.041890 282.072269 14.069490 109.766783    8.553682 224.894491 12.614897
EPOCH=6 4155.295180 8767.729167 347.000355 16.050343 995.784263 26.607640 264.313168    13.997479 442.170245 18.191059
EPOCH=7 3740.208430 5448.924727 415.878051 18.461288 181.907499 11.656418 246.155722    13.166414 433.687635 17.042779
EPOCH=8 4528.493080 67627.210317 3832.586361 50.377995 5695.053048 61.800994 2777.929237    46.996487 2483.623419 46.562377
EPOCH=9 3651.211018 2288.940104 450.389

In [ ]:
# G-CNN
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
          nn.Linear(256*10, 10),
        )

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    output = self.encoder(input)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 12743.737293 39590.204365 3935.367851 61.738008 4109.781138 63.079644 4280.498127    64.516863 4426.276500 65.818612
EPOCH=2 1952.249801 40987.028770 4246.991412 63.009406 4198.339726 62.619859 4353.838281    64.058166 4483.885851 65.549625
EPOCH=3 2209.218721 14683.310764 2744.568074 51.135515 2517.025918 49.110526 2699.581548    50.978034 2894.298772 51.945986
EPOCH=4 2732.989978 2322.607267 534.482407 19.355042 665.848796 23.271873 370.637835    16.966094 235.291304 13.410157
EPOCH=5 2126.484753 11674.061756 334.229836 15.278382 511.574611 20.206872 607.677755    23.377556 444.771816 17.733606
EPOCH=6 1323.281538 10539.186260 2762.447445 49.550410 1097.452305 30.041689 68.300176    6.447385 60.719792 6.388438
EPOCH=7 824.196668 2002.729570 598.418803 21.970505 86.259277 7.698534 97.556134    8.314916 63.065268 6.734463
EPOCH=8 851.337028 1699.702474 159.729804 10.334446 339.489391 16.144755 591.832699    22.727270 309.596468 14.789944
EPOCH=9 595.427631 3887.365823 203.92765

In [ ]:
# GRU
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.avg_pool = nn.AvgPool1d(kernel_size=7)
    self.batch_norm = nn.BatchNorm1d(224*224)
    self.gru = nn.GRU(1568, 256, batch_first=True, bidirectional=False, num_layers=1, dropout=0.6)
    self.out = nn.Linear(256, 10)

  def forward(self, input, hidden):
    input = input.permute(0, 2, 1)
    input = self.avg_pool(input)
    input = input.squeeze(2)
    input = self.batch_norm(input)

    input = input.view(-1, 32, 1568)
    gru_out, _ = self.gru(input)
    output = gru_out[:, -1, :]
    output = self.out(output)
    return output, 1
  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


EPOCH=1 127047.513999 122668.986111 10477.915427 98.324017 10773.074950 99.847867 10900.845350    100.522431 11381.326190 102.825484
EPOCH=2 106924.680776 103948.414683 8738.102703 89.030127 9002.901166 90.543148 9117.666815    91.216805 9552.349479 93.504575
EPOCH=3 90138.232474 88128.505952 7280.927734 80.430349 7517.003100 81.926807 7618.846664    82.592669 8009.733755 84.855125
EPOCH=4 75982.083003 74474.885913 6034.379967 72.300732 6242.994271 73.742457 6332.687004    74.399959 6681.835231 76.632003
EPOCH=5 63847.017857 63106.929563 5008.162023 64.920283 5190.726606 66.271799 5268.944869    66.870462 5578.276345 69.057558
EPOCH=6 53719.895282 53281.650794 4131.963399 57.926974 4289.625744 59.191021 4356.222281    59.656684 4627.141391 61.788299
EPOCH=7 45176.754960 45086.001984 3410.534858 51.512183 3545.166474 52.682902 3601.726279    52.974444 3837.247914 55.026281
EPOCH=8 38085.868965 38328.414683 2825.964102 45.689858 2939.215313 46.761759 2985.880018    46.884620 3188.101335 

In [ ]:
# Bi-GRU
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.avg_pool = nn.AvgPool1d(kernel_size=7)
    self.batch_norm = nn.BatchNorm1d(224*224)
    self.gru = nn.GRU(1568, 256, batch_first=True, bidirectional=True, num_layers=1, dropout=0.6)
    self.out = nn.Linear(256*2, 10)

  def forward(self, input, hidden):
    input = input.permute(0, 2, 1)
    input = self.avg_pool(input)
    input = input.squeeze(2)
    input = self.batch_norm(input)

    input = input.view(-1, 32, 1568)
    gru_out, _ = self.gru(input)
    output = gru_out[:, -1, :]
    output = self.out(output)
    return output, 1
  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


EPOCH=1 122504.721340 111644.103175 9544.843378 93.722100 9723.277183 94.716909 10053.854241    96.461082 10222.794916 97.289400
EPOCH=2 92443.986883 83694.150794 7002.107174 79.059605 7145.848332 80.014491 7426.470536    81.774751 7562.816257 82.538653
EPOCH=3 68637.865079 61857.728175 5001.406424 65.163847 5111.238641 65.979856 5336.092721    67.678382 5448.249206 68.427137
EPOCH=4 50056.335207 45537.315476 3512.865335 52.543936 3594.261732 53.211395 3763.457791    54.508363 3849.633042 55.167035
EPOCH=5 36398.791171 33833.308036 2475.311268 41.403080 2529.926239 41.864524 2647.398401    42.684909 2710.855954 42.990899
EPOCH=6 26645.731233 25837.251984 1810.971778 32.221114 1843.713382 32.521314 1915.434658    32.912113 1960.564396 33.006952
EPOCH=7 20664.406278 21103.552083 1446.279219 28.240898 1465.543856 28.429943 1504.599434    28.757518 1538.372194 29.102451
EPOCH=8 16668.152585 16842.432788 1136.280143 23.254536 1152.349885 23.457417 1182.025812    23.670228 1208.182217 23.886

In [ ]:
# M1
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
        )
    self.dropout = nn.Dropout(0.5)

    self.gru = nn.GRU(self.hidden_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.out = nn.Linear(self.hidden_size*2, 1)

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    encoder_outputs = self.encoder(input)
    encoder_outputs = self.dropout(encoder_outputs)
    print
    encoder_outputs = encoder_outputs.view(-1, 10, 256)
    
    hidden, _ = self.gru(encoder_outputs)
    output = self.out(hidden)
    output = output.squeeze(2)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 103627.627094 89095.571429 7455.358829 81.507042 7521.314298 81.952413 7781.390340    83.570301 8031.393849 84.982208
EPOCH=2 70018.101687 61462.852183 4916.933625 64.225171 4967.350558 64.584343 5171.004111    66.133268 5371.763176 67.545051


KeyboardInterrupt: ignored

In [ ]:
# SMAML
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),
        )
    self.linear1 = nn.Linear(7*7*72, self.hidden_size)
    self.batch_norm = nn.BatchNorm1d(self.hidden_size)

    # Decoder
    self.linear2 = nn.Linear(224*224, self.hidden_size*2)
    self.dropout = nn.Dropout(0.8)

    self.gru = nn.GRU(self.hidden_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.out = nn.Linear(self.hidden_size*2, 1)

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    encoder_outputs = self.encoder(input)
    encoder_outputs = encoder_outputs.view(-1, 7, 7*7*72)
    encoder_outputs = self.dropout(self.linear1(encoder_outputs))

    # Decoder with attention
    input = input.view(-1, 7, 224*224)
    embed = torch.tanh(self.linear2(input))
    embed = self.dropout(embed)

    hidden = hidden[: len(embed)]
    attn_weights = F.softmax(torch.bmm(hidden, embed.permute(0, 2, 1)), dim=2)
    attn_applied = F.relu(torch.bmm(attn_weights, encoder_outputs))
    attn_applied = self.batch_norm(attn_applied.permute(0, 2, 1))
    attn_applied = attn_applied.permute(0, 2, 1)

    hidden, _ = self.gru(attn_applied)
    output = self.out(hidden)
    output = output.squeeze(2)
    return output, attn_weights

  def initHidden(self):
    return torch.randn(16, self.output_size, self.hidden_size*2, device=device)


model = Model()
train(model, lamb=0.1, max_iter=200)

EPOCH=1 118463.774802 94043.333333 9043.519370 91.062185 8056.841890 85.764545 7893.786694    84.488335 9745.336086 94.940172
EPOCH=2 72697.528715 58860.495040 5608.949975 69.858468 4744.764661 63.420966 4684.373264    62.409243 6150.306250 73.681959
EPOCH=3 46295.298969 38620.038690 3880.039140 56.951715 3053.175068 48.634580 2941.057448    46.629834 4283.381926 60.217296
EPOCH=4 29705.876605 26284.920139 2722.508411 46.462668 1977.620784 36.967849 1804.495081    32.653568 3210.292361 51.750891
EPOCH=5 20281.908771 19649.910218 1807.378550 34.175558 1284.360585 26.379551 1214.437168    24.479630 2341.745094 39.958682
EPOCH=6 15584.617746 14443.286706 1341.475879 29.647570 1056.135941 24.905594 946.581402    22.411608 1512.782896 31.734637
EPOCH=7 12909.148658 18352.522073 4149.826426 56.323244 3536.084253 49.149991 3267.630784    44.967031 4501.612754 59.874736
EPOCH=8 11628.009018 11008.171627 1093.026264 28.314322 865.109255 23.918081 812.211672    22.640930 1291.839621 30.877517
EP

KeyboardInterrupt: ignored

In [ ]:
def pred(model, val_loader, test_loader):
  model.eval()
  hidden = model.initHidden()
  pred_label = []
  mse = 0.0
  weights = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, weight = model(inputs, hidden)
    weights.append(weight.to('cpu').detach().numpy())
    pred_label.append(output.to('cpu').detach().numpy())
    step = 0
    labels = labels[:, step]
    output = output[:, step]
    mse_sum = torch.sum(torch.pow((output - labels), 2))
    mse += mse_sum.item()
  mse = mse/315
  return np.vstack(pred_label), mse, np.array(weights)

preds, mse, weights = pred(model, val_loader, test_loader)
print(mse)
for k, i in enumerate(preds):
  print(','.join([str(k) for k in list(i)]))

torch.cuda.empty_cache()

301.031153312562
67.92358,69.876335,70.66485,70.82065,70.22916,71.713646,71.50754,71.13899,74.98229,75.46991
77.708534,76.96415,76.63735,76.083664,83.72402,86.329346,84.43171,86.252045,79.54052,80.48531
66.88258,63.362286,65.71035,65.65867,66.35354,66.270195,67.031624,68.09753,72.02675,72.84423
69.19776,66.36722,67.028656,66.32448,66.88923,65.03648,65.23697,66.31954,66.984886,65.884155
71.759026,77.98195,78.42031,78.73118,78.997765,79.24675,79.34009,80.354744,80.70208,82.50687
72.83006,73.10901,72.99141,72.89564,72.59561,73.06616,73.45066,73.41811,74.432175,75.27235
81.96655,87.12719,85.36069,85.49302,85.55965,85.64179,87.09851,84.55969,84.375404,86.47033
71.18769,70.435394,70.33931,69.89865,70.08787,69.96808,69.78139,70.58192,72.01757,72.7356
75.52231,75.11677,74.18165,74.92341,77.933624,76.703705,76.8431,75.48066,76.693016,77.77889
76.88742,82.5917,83.304634,78.25418,77.524956,77.886055,78.27253,78.57934,82.09176,82.98129
77.785286,88.027596,80.82183,80.04474,79.89751,80.68089,82.889

In [ ]:
for i in weights[10][10]:
  print(','.join([str(round(x, 4)) for x in i]))

0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.9974,0.0026,0.0,0.0
0.0,0.0169,0.0,0.0,0.9831,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.0,0.7454,0.0,0.0108,0.0,0.0,0.2438
1.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.0,0.0,1.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.0,0.0,0.0,0.0,0.0,0.0,1.0
